In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import time
%matplotlib inline

In [2]:
fer = pd.read_csv("C:/Users/Teja/Desktop/teja/Education/Projects/FER/fer2013/fer2013.csv")

In [3]:
fer.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [4]:
width, height = 48, 48
faces_pixels = fer['pixels'].tolist()
faces = []

for face in faces_pixels:
    face = face.split(" ")
    face = np.asarray(face).reshape(width, height)
    faces.append(face.astype('float32'))


In [5]:
faces = np.array(faces)
faces = np.expand_dims(faces, -1)
faces.shape

(35887, 48, 48, 1)

In [6]:
emotion = pd.get_dummies(fer['emotion']).as_matrix()
# emotion.shape

C:\Users\Teja\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [7]:
from sklearn.model_selection import train_test_split


X_train, X_test, Y_train, Y_test = train_test_split(faces, emotion, test_size = 0.1, random_state = 123)
# X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state = 123)

In [8]:
x = tf.placeholder(tf.float32, shape = [None, 48*48], name = "X")
x = tf.reshape(x, [-1, 48, 48, 1])

y = tf.placeholder(tf.float32, shape = [None, 7], name = "Y")
y_image = tf.argmax(y, axis = 1)

##  Convolution Layer

In [9]:
def new_conv(input, w, b, stride=1):
    layer = tf.nn.conv2d(input, w, strides = [1, stride, stride, 1], padding= "SAME")
    layer = tf.nn.bias_add(layer, b)
    layer = tf.nn.relu(layer)
    return layer

## Pooling Layer

In [10]:
def pool_layer(input, stride = 2):
    layer = tf.nn.max_pool(input, ksize=[1, stride, stride, 1], strides = [1, stride, stride, 1], padding = "SAME")
    return layer

In [11]:
# Initializing parameters
weights = {
    'W0' : tf.Variable(tf.truncated_normal([5, 5, 1, 64], stddev = 0.05), name = 'w0'),
    'W1' : tf.Variable(tf.truncated_normal([5, 5, 64, 128], stddev = 0.05), name = 'w1'),
    'W2' : tf.Variable(tf.truncated_normal([5, 5, 128, 256], stddev = 0.05), name = 'w2'),
    'W3' : tf.Variable(tf.truncated_normal([6*6*256, 4096], stddev = 0.05), name = 'w3'),
    'W4' : tf.Variable(tf.truncated_normal([4096, 1024], stddev = 0.05), name = 'w4'),
    'W5' : tf.Variable(tf.truncated_normal([1024, 256], stddev = 0.05), name = 'w5'),
    'W6' : tf.Variable(tf.truncated_normal([256, 7], stddev = 0.05), name = 'w6')
}
biases = {
    'B0' : tf.Variable(tf.constant(0.05, shape= [64]), name = 'b0'),
    'B1' : tf.Variable(tf.constant(0.05, shape= [128]), name = 'b1'),
    'B2' : tf.Variable(tf.constant(0.05, shape= [256]), name = 'b2'),
    'B3' : tf.Variable(tf.constant(0.05, shape= [4096]), name = 'b3'),
    'B4' : tf.Variable(tf.constant(0.05, shape= [1024]), name = 'b4'),
    'B5' : tf.Variable(tf.constant(0.05, shape= [256]), name = 'b5'),
    'B6' : tf.Variable(tf.constant(0.05, shape= [7]), name = 'b6')
}


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
weights1 = {
    'W0' : tf.Variable(tf.truncated_normal([3, 3, 1, 64], stddev = 0.05), name = 'w0'),
    'W1' : tf.Variable(tf.truncated_normal([3, 3, 64, 64], stddev = 0.05), name = 'w0'),
    'W2' : tf.Variable(tf.truncated_normal([3, 3, 64, 128], stddev = 0.05), name = 'w0'),
    'W3' : tf.Variable(tf.truncated_normal([3, 3, 128, 128], stddev = 0.05), name = 'w0'),
    'W4' : tf.Variable(tf.truncated_normal([3, 3, 128, 256], stddev = 0.05), name = 'w0'),
    'W5' : tf.Variable(tf.truncated_normal([3, 3, 256, 256], stddev = 0.05), name = 'w0'),
    'W6' : tf.Variable(tf.truncated_normal([3, 3, 256, 512], stddev = 0.05), name = 'w0'),
    'W7' : tf.Variable(tf.truncated_normal([3, 3, 512, 512], stddev = 0.05), name = 'w0'),
    'W8' : tf.Variable(tf.truncated_normal([3*3*512, 512], stddev = 0.05), name = 'w0'),
    'W9' : tf.Variable(tf.truncated_normal([512, 256], stddev = 0.05), name = 'w0'),
    'W10' : tf.Variable(tf.truncated_normal([256, 128], stddev = 0.05), name = 'w0'),
    'W11' : tf.Variable(tf.truncated_normal([128, 7], stddev = 0.05), name = 'w0')
}

biases1 = {
    'B0' : tf.Variable(tf.constant(0.05, shape= [64]), name = 'b0'),
    'B1' : tf.Variable(tf.constant(0.05, shape= [64]), name = 'b1'),
    'B2' : tf.Variable(tf.constant(0.05, shape= [128]), name = 'b2'),
    'B3' : tf.Variable(tf.constant(0.05, shape= [128]), name = 'b3'),
    'B4' : tf.Variable(tf.constant(0.05, shape= [256]), name = 'b4'),
    'B5' : tf.Variable(tf.constant(0.05, shape= [256]), name = 'b5'),
    'B6' : tf.Variable(tf.constant(0.05, shape= [512]), name = 'b6'),
    'B7' : tf.Variable(tf.constant(0.05, shape= [512]), name = 'b7'),
    'B8' : tf.Variable(tf.constant(0.05, shape= [512]), name = 'b8'),
    'B9' : tf.Variable(tf.constant(0.05, shape= [256]), name = 'b9'),
    'B10' : tf.Variable(tf.constant(0.05, shape= [128]), name = 'b10'),
    'B11' : tf.Variable(tf.constant(0.05, shape= [7]), name = 'b11')
}

In [12]:
# Building Convolution Nueral Network
def CNN(x, weights, biases):
    # First Convolution Layer
    layer1 = new_conv(x, weights['W0'], biases['B0'])
    layer1 = pool_layer(layer1)
    
    #Second Convolution Layer
    layer2 = new_conv(layer1, weights['W1'], biases['B1'])
    layer2 = pool_layer(layer2)
    
    # Third Convolution Layer
    layer3  = new_conv(layer2, weights['W2'], biases['B2'])
    layer3 = pool_layer(layer3)
    
    # Flattening the Convolution Layer
    layer4 = tf.reshape(layer3, shape = [-1, weights['W3'].get_shape().as_list()[0]])
    
    # Dropout 
    layer4 = tf.nn.dropout(layer4, rate= .3, seed = 1)
    
    # Fully Connected layer 1
    fc1 = tf.add(tf.matmul(layer4, weights['W3']), biases['B3'])
    fc1 = tf.nn.relu(fc1)
    
    #Dropout
    fc1 = tf.nn.dropout(fc1, rate= .3, seed = 2) 
    
    # Fully Connected Layer 2
    fc2 = tf.add(tf.matmul(fc1, weights['W4']), biases['B4'])
    fc2 = tf.nn.relu(fc2)
    
    # Dropout
    fc2 = tf.nn.dropout(fc2, rate= .3, seed = 3)
    
    # Fully Connected layer 3
    fc3 = tf.add(tf.matmul(fc2, weights['W5']), biases['B5'])
    fc3 = tf.nn.relu(fc3)
    
    # Dropout
    fc3 = tf.nn.dropout(fc3, rate= .3, seed = 4) 
    
    # Fully Connected Layer 4
    out  = tf.add(tf.matmul(fc3, weights['W6']), biases['B6'])
#     out = tf.nn.softmax(out)
    
    return out

In [13]:
pred = CNN(x, weights, biases)

learning_rate = 0.001
batch_size = 128
epochs = 100

In [14]:
# Calculating Cost function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = pred, labels = y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [15]:
# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate).minimize(cost)

In [16]:
# Accuracy
matches = tf.equal(tf.argmax(tf.nn.softmax(pred), 1), y_image)
accuracy = tf.reduce_mean(tf.cast(matches, tf.float32))

In [ ]:
# Training model
saver = tf.train.Saver()
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    train_loss = []
    testing_loss = []
    train_accuracy = []
    testing_accuracy = []
    
    for i in range(epochs):
        start_time = time.time()
        loss, acc = 0, 0
        for batch in range(len(X_train)// batch_size):
            X_batch = X_train[batch * batch_size : min((batch+1) * batch_size, len(X_train))]
            Y_batch = Y_train[batch * batch_size : min((batch+1) * batch_size, len(Y_train))]
            
            # Running Optimizer
            opt = sess.run(optimizer, feed_dict = {x: X_batch, y: Y_batch})
            
            # Running cost function and cost
            loss += sess.run(cost, feed_dict = {x: X_batch, y: Y_batch})
            acc += sess.run(accuracy, feed_dict = {x: X_batch, y: Y_batch})
        loss /= int(len(X_train)/batch_size)
        acc /= int(len(X_train)/batch_size)
        test_loss, test_acc = sess.run([cost, accuracy], feed_dict = {x: X_test, y: Y_test})
        
        end_time = time.time()
        
        print("Epochs " + str(i+1) + " completed: " + "Time Usage {:.3f}".format(end_time - start_time))
        print("Accuracy:")
        print("\t* Training Accuracy:\t{:.6f}".format(acc))
        print("\t* Test Accuracy:\t{:.6f}".format(test_acc))
        print("Loss:")
        print("\t* Training Loss:\t{:.6f}".format(loss))
        print("\t* Test Loss:\t{:.6f}".format(test_loss))
        
        train_loss.append(loss)
        testing_loss.append(test_loss)
        train_accuracy.append(acc)
        testing_accuracy.append(test_acc)
        saver.save(sess, 'my_model')

In [ ]:
plt.plot(range(len(train_loss)), train_loss, 'b', label = "Training Loss")
plt.plot(range(len(testing_loss)), testing_loss, 'r', label = "Test Loss")
plt.title("Training Loss Vs Test Loss")
plt.xlabel("no of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(range(len(train_accuracy)), train_accuracy, 'b', label = "Training Accuracy")
plt.plot(range(len(testing_accuracy)), testing_accuracy, 'r', label = "Test Loss")
plt.title("Training Accuracy Vs Validation Accuracy")
plt.xlabel("no of Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
def CNN1(x, weights1, biases1):
    # First Convolution Layer
    layer1 = new_conv(x, weights1['W0'], biases1['B0'])
    layer1 = new_conv(layer1, weights1['W1'], biases1['B1'])
    layer1 = pool_layer(layer1)
    layer1 = tf.nn.dropout(layer1, rate= .3, seed = 143)
    
    # Second Convolution Layer
    layer2 = new_conv(layer1, weights1['W2'], biases1['B2'])
    layer2 = new_conv(layer2, weights1['W3'], biases1['B3'])
    layer2 = pool_layer(layer2)
    layer2 = tf.nn.dropout(layer2, rate= .3, seed = 143)
    
    # Third Convolution Layer
    layer3 = new_conv(layer2, weights1['W4'], biases1['B4'])
    layer3 = new_conv(layer3, weights1['W5'], biases1['B5'])
    layer3 = pool_layer(layer3)
    layer3 = tf.nn.dropout(layer3, rate= .3, seed = 143)
    
    # Fourth Convolution Layer
    layer4 = new_conv(layer3, weights1['W6'], biases1['B6'])
    layer4 = new_conv(layer4, weights1['W7'], biases1['B7'])
    layer4 = pool_layer(layer4)
    layer4 = tf.nn.dropout(layer4, rate= .3, seed = 143)
    
    # Flattening the layer4
    fc1 = tf.reshape(layer4, shape = [-1, weights1['W8'].get_shape().as_list()[0]])
    
    # Fully Connected layer 1
    fc1 = tf.add(tf.matmul(fc1, weights1['W8']), biases1['B8'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, rate= .3, seed = 143)
    
    # Fully Connected layer 2
    fc2 = tf.add(tf.matmul(fc1, weights1['W9']), biases1['B9'])
    fc2 = tf.nn.relu(fc2)
    fc2 = tf.nn.dropout(fc2, rate= .3, seed = 143)
    
    # Fully Connected layer 3
    fc3 = tf.add(tf.matmul(fc2, weights1['W10']), biases1['B10'])
    fc3 = tf.nn.relu(fc3)
    fc3 = tf.nn.dropout(fc3, rate= .3, seed = 143)
    
    # Fully Connected layer 4
    fc4 = tf.add(tf.matmul(fc3, weights1['W11']), biases1['B11'])
    
    return fc4

In [ ]:
pred1 = CNN1(x, weights1, biases1)

In [ ]:
cost1 = tf.reduce_mean(tf.nn.softmax_cross_entropy_logits(logits= pred1, labels = y))

In [ ]:
optimizer1 = tf.train.AdamOptimizer(learning_rate =  learning_rate).minimize(cost1)

In [ ]:
matches1 = tf.equal(tf.argmax(pred1, 1), y_image)
accuracy1 = tf.reduce_mean(tf.cast(matches1, tf.float32))

In [ ]:
# Training model
saver = tf.train.Saver()
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    train_loss1 = []
    test_loss1 = []
    train_accuracy1 = []
    test_accuracy1 = []
    
    for i in range(epochs):
        start_time = time.time()
        loss, acc = 0, 0
        for batch in range(len(X_train)// batch_size):
            X_batch = X_train[batch * batch_size : min((batch+1) * batch_size, len(X_train))]
            Y_batch = Y_train[batch * batch_size : min((batch+1) * batch_size, len(Y_train))]
            
            # Running Optimizer
            opt = sess.run(optimizer1, feed_dict = {x: X_batch, y: Y_batch})
            
            # Running cost function and cost
            loss += sess.run(cost1, feed_dict = {x: X_batch, y: Y_batch})
            acc += sess.run(accuracy1, feed_dict = {x: X_batch, y: Y_batch})
        loss /= int(len(X_train)/batch_size)
        acc /= int(len(X_train)/batch_size)
        test_loss, test_acc = sess.run([cost1, accuracy1], feed_dict = {x: X_test, y: Y_test})
        
        end_time = time.time()
        
        print("Epochs " + str(i+1) + " completed: " + "Time Usage {:.3f}".format(end_time - start_time))
        print("Accuracy:")
        print("\t* Training Accuracy:\t{:.6f}".format(acc))
        print("\t* Test Accuracy:\t{:.6f}".format(test_acc))
        print("Loss:")
        print("\t* Training Loss:\t{:.6f}".format(loss))
        print("\t* Test Loss:\t{:.6f}".format(test_loss))
        
        train_loss1.append(loss)
        test_loss1.append(test_loss)
        train_accuracy1.append(acc)
        test_accuracy1.append(test_acc)
        saver.save(sess, "my_model")

In [ ]:
plt.plot(range(len(train_loss1)), train_loss1, 'b', label = "Training Loss")
plt.plot(range(len(test_loss1)), test_loss1, 'r', label = "Test Loss")
plt.title("Training Loss Vs Test Loss")
plt.xlabel("no of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(range(len(train_accuracy1)), train_accuracy1, 'b', label = "Training Accuracy")
plt.plot(range(len(test_accuracy1)), test_accuracy1, 'r', label = "Test Loss")
plt.title("Training Accuracy Vs Validation Accuracy")
plt.xlabel("no of Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
sess = tf.Session()
res = tf.train.import_meta_graph("my_model.meta")
res.restore(sess, tf.train.latest_checkpoint('./'))
graph = tf.get_defaut_graph()

In [ ]:
weights = {
    W0 = graph.get_tensor_by_name("w0:0"),
    W1 = graph.get_tensor_by_name("w1:0"),
    W2 = graph.get_tensor_by_name("w2:0"),
    W3 = graph.get_tensor_by_name("w3:0"),
    W4 = graph.get_tensor_by_name("w4:0"),
    W5 = graph.get_tensor_by_name("w5:0"),
    W6 = graph.get_tensor_by_name("w6:0")
}
biases = {
    B0 = graph.get_tensor_by_name("b0:0"),
    B1 = graph.get_tensor_by_name("b1:0"),
    B2 = graph.get_tensor_by_name("b2:0"),
    B3 = graph.get_tensor_by_name("b3:0"),
    B4 = graph.get_tensor_by_name("b4:0"),
    B5 = graph.get_tensor_by_name("b5:0"),
    B6 = graph.get_tensor_by_name("b6:0")
}

In [ ]:
weights1 = {
    W0 = graph.get_tensor_by_name("w0:0"),
    W1 = graph.get_tensor_by_name("w1:0"),
    W2 = graph.get_tensor_by_name("w2:0"),
    W3 = graph.get_tensor_by_name("w3:0"),
    W4 = graph.get_tensor_by_name("w4:0"),
    W5 = graph.get_tensor_by_name("w5:0"),
    W6 = graph.get_tensor_by_name("w6:0"),
    W7 = graph.get_tensor_by_name("w7:0"),
    W8 = graph.get_tensor_by_name("w8:0"),
    W9 = graph.get_tensor_by_name("w9:0"),
    W10 = graph.get_tensor_by_name("w10:0"),
    W11 = graph.get_tensor_by_name("w11:0")
}
biases1 = {
    B0 = graph.get_tensor_by_name("b0:0"),
    B1 = graph.get_tensor_by_name("b1:0"),
    B2 = graph.get_tensor_by_name("b2:0"),
    B3 = graph.get_tensor_by_name("b3:0"),
    B4 = graph.get_tensor_by_name("b4:0"),
    B5 = graph.get_tensor_by_name("b5:0"),
    B6 = graph.get_tensor_by_name("b6:0"),
    B7 = graph.get_tensor_by_name("b7:0"),
    B8 = graph.get_tensor_by_name("b8:0"),
    B9 = graph.get_tensor_by_name("b9:0"),
    B10 = graph.get_tensor_by_name("b10:0"),
    B11 = graph.get_tensor_by_name("b11:0")
}

In [ ]:
emotion_dict = {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Sad", 5: "Surprise", 6: "Neutral"}